In [23]:
!pip install requests

In [11]:
import pandas as pd
import torch
from torch import nn
import torchvision.models as models
import torch.nn as nn
from collections import OrderedDict
from transformers import BertModel, BertTokenizer
import requests


In [ ]:
class BertResNet(nn.Module):
  """Neural network that makes use of BERT and ResNet architectures"""
  def __init__(self):
      super(BertResNet, self).__init__()

      self.resnet18 = models.resnet18(pretrained=True)
      n_inputs = self.resnet18.fc.in_features

      classifier = nn.Sequential(OrderedDict([
          ('fc1', nn.Linear(n_inputs, 512))
      ]))

      self.resnet18.fc = classifier

      self.token = BertTokenizer.from_pretrained('bert-base-uncased')
      self.bert = BertModel.from_pretrained('bert-base-uncased')

      self.classification = nn.Linear(512 + 768, 1)

  def forward(self, img, text):
        "Combined forward pass on img and text."
        res_emb = self.resnet18(img)
        bert_token = self.token(text)
        bert_emb = self.bert(bert_token)

        combined = torch.cat(res_emb,
                              bert_emb, dim=1)
        out = self.classification(combined)
        return out

In [12]:
multi_modal = BertResNet()

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
train_df.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,6,2020-11-15 16:01:08,525,An 85-year-old primary school in Shanghai has ...,cnni,[Video(thumbnailUrl='https://pbs.twimg.com/amp...,cnn


In [14]:
media = train_df["media"].values
media[:3]

array(["[Photo(previewUrl='https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=large')]",
       "[Photo(previewUrl='https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=large')]",
       "[Photo(previewUrl='https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?format=jpg&name=large')]"],
      dtype=object)

In [22]:
media_links = []
for i in media:
  media_links.append(i.split("'")[1])
len(media_links)

208801

In [27]:
def check_link(url):
    try:
        response = requests.head(url, timeout=10)
        if response.status_code == 200:
            return 1
        else:
            return 0
    except requests.RequestException as e:
        return f"An error occurred while checking the link {url}: {e}"

good_links = 0
broken_links = 0
cnt = 0

for link in media_links[:1000]:
  cnt += 1
  print(cnt)
  num = check_link(link)
  if num == 1:
    good_links += 1
  elif num == 0:
    broken_links += 1

good_links, broken_links

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


(987, 13)